In [ ]:
import httpx
import os
cookies = {'session': os.environ['AOC_SESSION']}
res = httpx.get('https://adventofcode.com/2025/day/5/input', cookies=cookies)

In [ ]:
instructions = list(res.iter_lines())

In [ ]:
sample_input_fresh, sample_input_available = '''3-5
10-14
16-20
12-18

1
5
8
11
17
32'''.split('\n\n')

sample_input_fresh = sample_input_fresh.splitlines()
sample_input_available = sample_input_available.splitlines()

In [ ]:
list(map(lambda x: list(map(int, x.split('-'))), sample_input_fresh))


[[3, 5], [10, 14], [16, 20], [12, 18]]

In [ ]:
5 in range(3,5+1)

True

Ok, so I actually don't want 3,5 in a list like this, I want to convert this into a range the goes from 3-6 so that 5 will be inclusive.  

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You're right! Since `range(3, 6)` would give you 3, 4, 5 (making 5 inclusive), you need to add 1 to the end value.

You could modify your lambda like this:

```python
lambda x: range(*map(int, x.split('-')))  # This gives range(3, 5) - not what you want
```

But to make the upper bound inclusive, you'd want:

```python
lambda x: range(int(x.split('-')[0]), int(x.split('-')[1]) + 1)
```

Would you like to try updating your code with this adjustment?

This is getting pretty clunky.  Can we do soemthing else that makes it easier to read?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Absolutely! A few options to consider:

1. **Define a small helper function** to handle the parsing and range creation - this makes your main code much cleaner
2. **Use tuple unpacking** with a split to make it more readable
3. **Use a list comprehension** instead of `map` with a lambda

Which approach appeals to you? Or would you like me to show a quick example of one of them?

I don't really want to think about this right now, can you show me all of these options written out in code?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Sure thing! Here are all three approaches:

**1. Helper function:**
```python
def parse_range(s):
    start, end = map(int, s.split('-'))
    return range(start, end + 1)

[parse_range(x) for x in sample_input_fresh]
```

**2. Tuple unpacking in a list comprehension:**
```python
[range(int(a), int(b) + 1) for a, b in (x.split('-') for x in sample_input_fresh)]
```

**3. Simpler list comprehension with inline unpacking:**
```python
[range(s, e + 1) for s, e in [map(int, x.split('-')) for x in sample_input_fresh]]
```

I'd personally lean toward option 1 - it's the most readable and you can reuse `parse_range` elsewhere. But pick whichever feels clearest to you!

### Part 1

In [ ]:
def parse_range(s):
    start, end = map(int, s.split('-'))
    return range(start, end + 1)

In [ ]:
fresh_ranges = [parse_range(x) for x in sample_input_fresh]

In [ ]:
fresh=0
for ingredient in map(int, sample_input_available):
    is_fresh = False
    for fresh_range in fresh_ranges: 
        print(ingredient, fresh_range)
        if ingredient in fresh_range: 
            is_fresh = True
    if is_fresh: 
        fresh+=1
print(fresh)

1 range(3, 6)
1 range(10, 15)
1 range(16, 21)
1 range(12, 19)
5 range(3, 6)
5 range(10, 15)
5 range(16, 21)
5 range(12, 19)
8 range(3, 6)
8 range(10, 15)
8 range(16, 21)
8 range(12, 19)
11 range(3, 6)
11 range(10, 15)
11 range(16, 21)
11 range(12, 19)
17 range(3, 6)
17 range(10, 15)
17 range(16, 21)
17 range(12, 19)
32 range(3, 6)
32 range(10, 15)
32 range(16, 21)
32 range(12, 19)
3


In [ ]:
fresh, available = res.text.split('\n\n')


In [ ]:
fresh = fresh.splitlines()
available = available.splitlines()

fresh_ranges = [parse_range(x) for x in fresh]

In [ ]:
fresh=0
for ingredient in map(int, available):
    is_fresh = False
    for fresh_range in fresh_ranges: 
        # print(ingredient, fresh_range)
        if ingredient in fresh_range: 
            is_fresh = True
    if is_fresh: 
        fresh+=1
print(fresh)

874


### Part 2

In [ ]:
set(range(1,4))|set(range(2,7))

{1, 2, 3, 4, 5, 6}

How would somebody append two sets together?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

For sets, you'd use the **union operator `|`** or the `.union()` method:

```python
set(range(1,4)) | set(range(5,7))
```

or

```python
set(range(1,4)).union(range(5,7))
```

Both will give you `{1, 2, 3, 5, 6}`.

In [ ]:
full_fresh_list = set()
for fresh_range_set in map(set, fresh_ranges):
    full_fresh_list|=fresh_range_set
len(full_fresh_list)

We need to somehow make this more performant. I have tried running the code and it spins for a very long time.  Any ideas?  Basically we have ranges and we need to find out how many unique values there are total. 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

The problem with converting ranges to sets is that if a range spans millions of numbers, you're creating millions of elements in memory!

A more efficient approach: instead of storing every individual number, think about **merging overlapping ranges** first. Once you have non-overlapping ranges, you can just sum up their lengths.

For example, if you have `range(1, 100)` and `range(50, 150)`, instead of creating 199 set elements, you could merge them into a single range `(1, 150)` and know the count is 149.

Does that make sense as an approach? Would you like to think through how to merge overlapping ranges, or would you like some hints?

In [ ]:
len(range(1,5))

4

Can you give me some hints?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Sure! Here's a hint to get started:

**Step 1: Sort the ranges** by their start value. This makes it much easier to detect overlaps since you only need to compare each range with the one before it.

**Step 2:** Walk through the sorted ranges. If the current range overlaps with (or touches) the previous one, merge them. Otherwise, start tracking a new range.

Two ranges overlap or touch if the start of the second is ≤ the end of the first (plus 1 for touching).

Try sorting your ranges first - what would you use as the sort key?

In [ ]:
sorted_ranges = sorted(fresh_ranges, key=lambda x: x.start)

In [ ]:
sorted_ranges

[range(3438495999024, 8748904577027),
 range(3438495999024, 3438495999025),
 range(13786664510155, 16801037686314),
 range(16801037686314, 16801037686315),
 range(23447989968748, 26150400337835),
 range(23447989968748, 26150400337835),
 range(32214673183952, 38856090557944),
 range(32924986374848, 36523931641409),
 range(40791986810555, 47370735506062),
 range(44133102501668, 47370735506062),
 range(52980365483722, 52980365483723),
 range(52980365483723, 59670119402836),
 range(60705168430289, 62694157540488),
 range(61959786002648, 63594086975230),
 range(63245729792531, 65187271227139),
 range(64627107934678, 66292153896232),
 range(65642977211556, 67444854661476),
 range(66836057276081, 69016933813235),
 range(67920735132186, 70036372437997),
 range(71433048213389, 75029841021092),
 range(75029841021092, 77676559008342),
 range(81573181850686, 86374068906215),
 range(86374068906216, 88141946429960),
 range(91397026032347, 98893982760231),
 range(98893982760230, 98893982760231),
 ran

Ok, I have it sorted now and I see the first two should probably be merged.  Should I just combine them and put the combined range in a new list?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Exactly! That's the right idea. You'd walk through the sorted ranges and:

1. Start with the first range as your "current" merged range
2. For each subsequent range, check if it overlaps or touches the current one
3. If yes → extend the current range's end (if needed)
4. If no → append the current range to your result list and start a new "current" range

One thing to watch: when merging, the new end should be the **max** of both ends (since sometimes a later range is fully contained within an earlier one).

Want to give the loop a try?

so if I have range1 and range2, I just need to see if range1.end is greater than range2 start? If not, I should add range1 to the completed_ranges list and then repeat the process for range2?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Yes, you've got the core logic! One small tweak: you'll want to check if `range1.stop >= range2.start` (greater than **or equal**) to catch ranges that touch but don't overlap.

For example, `range(3, 6)` and `range(6, 10)` should merge into `range(3, 10)` since they're contiguous.

And don't forget: when they *do* overlap, your new merged range's end should be `max(range1.stop, range2.stop)` in case range2 is contained inside range1.

Give it a try!

In [ ]:
finished_ranges = []
curr_range = None
for r in sorted_ranges: 
    if curr_range is None: 
        curr_range = r
        continue
    if curr_range.stop >= r.start: 
        curr_range = range(curr_range.start, max(curr_range.stop, r.stop))
    else: 
        finished_ranges.append(curr_range)
        curr_range=r
if curr_range is not None: 
    finished_ranges.append(curr_range)
    curr_range=None


In [ ]:
len(finished_ranges)

81

In [ ]:
sum([len(fr) for fr in finished_ranges])

348548952146313

Can you help me debug this? I submitted my answer to advent of code, but my answer is apparently incorrect

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

I see the bug! Look at your `else` block:

```python
else: 
    finished_ranges.append(curr_range)
    curr_range=None  # <-- here's the problem
```

When a range doesn't overlap, you're setting `curr_range = None`. But what about `r` itself? It's getting skipped! On the next iteration, `curr_range` is `None`, so you just grab the *next* range, and the one that didn't overlap is lost.

What should `curr_range` be set to instead?

Should r be set to the new curr_range?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Almost - it's the other way around! `curr_range` should be set to `r`:

```python
else:
    finished_ranges.append(curr_range)
    curr_range = r  # Start tracking the non-overlapping range
```

That way, the range that didn't overlap becomes the new "current" range you're trying to extend.

I'm wondering if this will be a performance hit.  Should I build up a full list and then worry about compressing them down into a set?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Good question to think about! A few things to consider:

1. **`|=` on sets is in-place** (like `.update()`), so you're not creating a brand new set each iteration - it's modifying `full_fresh_list` directly.

2. **Building a list first then converting** would use more memory (storing duplicates) and then still need to dedupe at the end.

So your current approach should actually be pretty efficient! But if you're curious about the actual performance, you could always use `%%timeit` to compare approaches on your real data.

Is performance actually a concern for your input size, or more of a curiosity?

In [ ]:
full_fresh_list = set()
for fresh_range in fresh_ranges:
    full_fresh_list|=set(fresh_range)
len(full_fresh_list)

It seems to be running quite slow.  